In [2]:
import pandas as pd

## Cleaning the data

In [3]:
data_set = pd.read_csv("Full_data.csv")

original_size = len(data_set)

# Dropping unnecessary columns
drop_columns = ["Unnamed: 0", "data", "analysis_url", "id", "track_href", "type", "uri"]
data_set = data_set.drop(drop_columns, axis=1)

# Dropping invalid songs
null_elem = data_set.isnull().any(axis=1)
data_set = data_set[~null_elem]
data_set = data_set.reset_index(drop=True)
print(data_set.columns)
reduced_size = len(data_set)

reduction = (original_size-reduced_size)/original_size
print("Data set reduction = ", reduction)
data_set

Index(['author', 'lyrics', 'style', 'title', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature',
       'valence'],
      dtype='object')
Data set reduction =  0.2874721355327686


,author,lyrics,style,title,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Midian Lima,"Amanheceu,Nada pesquei,Parecia ser apenas mais...",eletrônica,Não Pare,0.277000,0.517,293593.0,0.590,0.000000,4.0,0.1130,-6.670,1.0,0.0393,129.800,4.0,0.129
1,Ed Sheeran,"I found a love for me,Darling, just dive right...",eletrônica,Perfect,0.163000,0.599,263400.0,0.448,0.000000,8.0,0.1060,-6.312,1.0,0.0232,95.050,3.0,0.168
2,Lila Downs,"Son las tres de la mañana,Dicen que pena un sa...",cumbia,Zapata Se Queda,0.388000,0.680,264013.0,0.640,0.000000,5.0,0.4080,-7.803,1.0,0.1340,81.974,4.0,0.775
3,Bronco,"Y yo que me la lleve al río,Creyendo que era s...",cumbia,Naila,0.678000,0.779,429273.0,0.605,0.005550,4.0,0.1850,-9.522,0.0,0.0386,110.008,4.0,0.571
4,Bronco,"Junte lo mas hermoso,Que vivi yo contigo,Los d...",cumbia,Nunca Voy a Olvidarte,0.500000,0.404,194667.0,0.833,0.000000,2.0,0.9670,-6.245,1.0,0.1360,171.969,4.0,0.706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31959,Forfun,"Larguei um dub no carro e fui dar um rolé,Com ...",hardcore,Good Trip,0.056400,0.302,169373.0,0.920,0.000000,2.0,0.4540,-3.566,1.0,0.0925,179.956,4.0,0.407
31960,CPM 22,"Peguei minhas coisas fui embora,Não queria mai...",hardcore,Tarde de Outubro,0.000330,0.355,139400.0,0.955,0.000000,1.0,0.1320,-3.045,0.0,0.0800,179.043,4.0,0.654
31961,Raimundos,"Foi, foi, foi, foi, foi mal aí, véi!,Se eu fal...",hardcore,Deixa Eu Falar,0.021700,0.596,282133.0,0.903,0.000000,4.0,0.6280,-2.908,0.0,0.1450,133.173,4.0,0.309
31962,Sleeping With Sirens,"Life is walking on the tight rope,With nothing...",hardcore,How It Feels To Be Lost,0.000202,0.372,218004.0,0.961,0.000002,8.0,0.0945,-2.407,1.0,0.2210,165.054,4.0,0.337


In [9]:
styles = data_set['style'].unique()


array(['eletrônica', 'cumbia', 'zouk', 'pop', 'zamba', 'bolero',
       'post-rock', 'alternativo', 'brega', 'country', 'blues', 'dance',
       'disco', 'cuarteto', 'trova', 'soul', 'salsa', 'ska', 'rockabilly',
       'reggaeton', 'reggae', 'tango', 'regional', 'rock', 'samba',
       'tecnopop', 'sertanejo', 'power-pop', 'ranchera', 'mpb',
       'mariachi', 'industrial', 'indie', 'house', 'infantil',
       'instrumental', 'jazz', 'pagode', 'mambo', 'psicodelia',
       'merengue', 'hardcore', 'grunge', 'funk', 'folk', 'emocore',
       'fado'], dtype=object)